# Inbound Auth

AgentCore Identity를 사용하면 AgentCore Runtime에서 agent나 tool을 호출하는 사용자 및 애플리케이션에 대한 인바운드 액세스(Inbound Auth)를 검증하거나 AgentCore Gateway 대상에 대한 액세스를 검증할 수 있습니다. 또한 agent에서 외부 서비스 또는 Gateway 대상으로의 안전한 아웃바운드 액세스(Outbound Auth)를 제공합니다. 기존 identity provider(예: Amazon Cognito)와 통합되며 독립적으로 또는 사용자를 대신하여(OAuth를 통해) 작동하는 agent에 대한 권한 경계를 적용합니다.

Inbound Auth는 AgentCore Runtime, AgentCore Gateway 또는 다른 환경에서 호스팅되는지 여부에 관계없이 agent 또는 tool을 호출하려는 호출자를 검증합니다. Inbound Auth는 IAM(SigV4 자격 증명) 또는 OAuth 인증과 함께 작동합니다.

기본적으로 Amazon Bedrock AgentCore는 IAM 자격 증명을 사용합니다. 즉, agent에 대한 사용자 요청은 사용자의 IAM 자격 증명으로 인증됩니다. OAuth를 사용하는 경우 AgentCore Runtime 리소스 또는 AgentCore Gateway 엔드포인트를 구성할 때 다음을 지정해야 합니다:

- OAuth discovery server Url — OpenID Connect discovery URL에 대해 ^.+/\.well-known/openid-configuration$ 패턴과 일치해야 하는 문자열

- Allowed audiences — JWT 토큰에 대해 허용되는 대상 목록

- Allowed clients — 허용되는 클라이언트 식별자 목록

AgentCore CLI를 사용하는 경우 **configure** 명령을 사용할 때 AgentCore Runtime에 대한 인증 유형(및 OAuth discovery server)을 지정할 수 있습니다. CreateAgentRuntime 작업 및 Amazon Bedrock AgentCore 콘솔을 사용할 수도 있습니다. Gateway를 생성하는 경우 CreateGateway 작업 또는 콘솔을 사용합니다.

사용자가 agent를 사용하기 전에 클라이언트 애플리케이션은 사용자가 OAuth authorizer로 인증하도록 해야 합니다. 클라이언트는 bearer 토큰을 받은 다음 호출 요청에서 agent에 전달합니다. agent는 수신 시 액세스를 허용하기 전에 인증 서버로 토큰을 검증합니다.


## 개요

이 튜토리얼에서는 01-AgentCore-runtime에서 배포한 agent를 수정하고 Cognito를 Identity provider로 사용하여 Inbound Auth를 구성합니다. 한 명의 사용자와 앱 클라이언트가 있는 Cognito User pool을 설정합니다. Cognito user pool을 사용하여 Inbound Auth와 함께 Amazon Bedrock AgentCore Runtime을 사용하여 기존 agent를 호스팅하는 방법을 배웁니다.

### 튜토리얼 아키텍처

<div style="text-align:center">
    <img src="images/inbound_auth_cognito.png" width="90%"/>
</div>

### 튜토리얼 세부 정보


| 정보                | 세부 사항                                                                        |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | Conversational                                                                   |
| Agent 유형          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성 요소  | AgentCore Runtime에서 agent 호스팅. Strands Agent 및 Amazon Bedrock Model 사용 |
| 튜토리얼 분야       | Cross-vertical                                                                   |
| 예제 복잡도         | Easy                                                                             |
| Inbound Auth        | Cognito                                                                          |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK and boto3                                     |



### 튜토리얼 주요 기능

* Amazon Cognito를 사용한 Inbound Auth와 함께 Amazon Bedrock AgentCore Runtime에서 Agent 호스팅
* Amazon Bedrock model 사용
* Strands Agent 사용


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 인증을 위한 Amazon Cognito 설정

앱 클라이언트와 한 명의 테스트 사용자가 있는 Cognito Userpool을 프로비저닝하겠습니다. 배포된 MCP 서버에 액세스하기 위한 JWT 토큰을 제공하기 위해 Amazon Cognito를 사용할 것입니다. 이를 위해 `utils` 스크립트의 `setup_cognito_user_pool` 지원 함수를 사용합니다.

참고: Cognito access_token은 2시간 동안만 유효합니다. access_token이 만료되면 `reauthenticate_user` 메서드를 사용하여 다른 access_token을 발급받을 수 있습니다.

In [ ]:
import sys
import os

# 현재 노트북의 디렉토리 경로 가져오기
# __file__이 없는 경우(노트북 환경) "."을 사용
current_dir = os.path.dirname(
    os.path.abspath("__file__" if "__file__" in globals() else ".")
)

# 상위 디렉토리(utils가 있는 위치)로 경로 설정
utils_dir = os.path.join(current_dir, "..")
utils_dir = os.path.abspath(utils_dir)

# sys.path에 추가하여 utils 모듈 import 가능하도록 설정
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

## AgentCore Runtime에 배포할 agent 준비

### Amazon Bedrock model을 사용하는 Strands Agent
01-AgentCore-runtime 튜토리얼에서 생성한 Strands Agent로 시작하여 Amazon Cognito를 Identity Provider로 사용하는 Inbound Auth로 구성하겠습니다.

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

# AgentCore Runtime 애플리케이션 초기화
app = BedrockAgentCoreApp()

# 날씨 조회 커스텀 tool 정의
@tool
def weather():
    """ Get weather """
    return "sunny"  # 더미 구현


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
# Bedrock model 인스턴스 생성
model = BedrockModel(
    model_id=model_id,
)
# Agent 생성: calculator와 weather tool을 사용
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

# AgentCore Runtime의 진입점 함수
@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    # response.message 구조에서 텍스트 추출
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore Runtime에 agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 인증 메커니즘을 구성하여 클라이언트가 agent와 통신하는 방법을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore runtime에 배포할 수 있습니다.

### AgentCore Runtime 배포 구성

다음으로 스타터 툴킷을 사용하여 엔트리포인트, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime 배포를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다.

**중요** - 이전 단계의 Cognito Discovery url과 Cognito App client id를 업데이트하세요.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
region

# Cognito 설정에서 discovery URL과 client ID 추출
discovery_url = cognito_config.get("discovery_url")

client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

# AgentCore Runtime 구성: Cognito JWT authorizer 설정
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,  # OpenID Connect discovery URL
            "allowedClients": [client_id],  # 허용된 클라이언트 ID 목록
        }
    },
)
response

## AgentCore 구성 검토

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore Runtime에 agent 시작

이제 docker 파일이 있으므로 agent를 AgentCore Runtime에 시작하겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인하겠습니다.

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
# 배포가 완료될 때까지 폴링
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)
status

### 인증 없이 AgentCore Runtime 호출

마지막으로 페이로드로 AgentCore Runtime을 호출할 수 있습니다. 다음 셀을 실행해 보면 **"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type"**라는 오류가 표시됩니다.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### 인증과 함께 AgentCore Runtime 호출

올바른 인증 토큰 유형으로 agent를 호출하겠습니다. 우리의 경우 Cognito access token이 됩니다. "**Provision a Cognito User Pool**" 셀에서 access token을 복사하세요.

In [ ]:
# Cognito access token 재발급 (2시간 유효)
bearer_token = reauthenticate_user(cognito_config.get("client_id"))
# bearer token을 사용하여 인증된 호출
invoke_response = agentcore_runtime.invoke(
    {"prompt": "How is the weather now?"}, bearer_token=bearer_token
)
invoke_response

## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리하겠습니다.

In [ ]:
from boto3.session import Session
import boto3

boto_session = Session()

agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
ecr_client = boto3.client("ecr", region_name=region)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split("/")[1], force=True
)

# 축하합니다!